![Pytorch](../../../pytorch_logo_2018.svg)

# Pytorch 中级篇（2）：深度残差网络（Deep Residual Networks）

>参考代码
>
>**yunjey的 [pytorch tutorial系列](https://github.com/yunjey/pytorch-tutorial/tree/master/tutorials/02-intermediate/deep_residual_network/main.py)**

## 深度残差网络 学习资源

>**论文原文**
>
>[Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)

>**Kaiming He的深度残差网络PPT**
>
>[秒懂！何凯明的深度残差网络PPT是这样的|ICML2016 tutorial](https://www.leiphone.com/news/201608/vhqwt5eWmUsLBcnv.html)

## Pytorch实现

根据原文【4.2. CIFAR-10 and Analysis】一节设计的针对数据集CIFAR-10的深度残差网络。

### 预处理

In [1]:
# 包
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# 设备配置
torch.cuda.set_device(1) # 这句用来设置pytorch在哪块GPU上运行
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 超参数设置
num_epochs = 80
learning_rate = 0.001

# 图像预处理模块
# 先padding ，再 翻转，然后 裁剪。数据增广的手段
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

### CIFAR-10 数据集

In [3]:
#  训练数据集
train_dataset = torchvision.datasets.CIFAR10(root='../../../data/cifar-10',
                                             train=True, 
                                             transform=transform,
                                             download=True)

# 测试数据集
test_dataset = torchvision.datasets.CIFAR10(root='../../../data/cifar-10',
                                            train=False, 
                                            transform=transforms.ToTensor())

# 数据加载器
# 训练数据加载器
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)
# 测试数据加载器
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

Files already downloaded and verified


### 深度残差网络模型设计

#### 3x3卷积层

In [4]:
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

#### 残差块（残差单元）（Residual  block）

In [5]:
# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

#### 残差网络搭建

In [6]:
# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[0], 2)
        self.layer3 = self.make_layer(block, 64, layers[1], 2)
        self.avg_pool = nn.AvgPool2d(8,ceil_mode=False) #  nn.AvgPool2d需要添加参数ceil_mode=False，否则该模块无法导出为onnx格式
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample)) # 残差直接映射部分
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

#### 实例化模型

In [7]:
# 实例化一个残差网络模型
model = ResNet(ResidualBlock, [2, 2, 2, 2]).to(device)

# 设置损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 用于更新参数组中的学习率
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

### 训练模型

In [8]:
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if ((i+1) % 100 == 0) and ((epoch+1) % 5 == 0):
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # 学习率衰减策略
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch [5/80], Step [100/500] Loss: 0.6474
Epoch [5/80], Step [200/500] Loss: 0.7043
Epoch [5/80], Step [300/500] Loss: 0.7472
Epoch [5/80], Step [400/500] Loss: 0.6662
Epoch [5/80], Step [500/500] Loss: 0.6378
Epoch [10/80], Step [100/500] Loss: 0.5786
Epoch [10/80], Step [200/500] Loss: 0.7229
Epoch [10/80], Step [300/500] Loss: 0.6183
Epoch [10/80], Step [400/500] Loss: 0.5043
Epoch [10/80], Step [500/500] Loss: 0.5799
Epoch [15/80], Step [100/500] Loss: 0.5295
Epoch [15/80], Step [200/500] Loss: 0.4475
Epoch [15/80], Step [300/500] Loss: 0.3896
Epoch [15/80], Step [400/500] Loss: 0.4869
Epoch [15/80], Step [500/500] Loss: 0.4973
Epoch [20/80], Step [100/500] Loss: 0.3953
Epoch [20/80], Step [200/500] Loss: 0.4542
Epoch [20/80], Step [300/500] Loss: 0.4003
Epoch [20/80], Step [400/500] Loss: 0.3863
Epoch [20/80], Step [500/500] Loss: 0.2813
Epoch [25/80], Step [100/500] Loss: 0.3860
Epoch [25/80], Step [200/500] Loss: 0.4341
Epoch [25/80], Step [300/500] Loss: 0.3384
Epoch [25/80], S

### 模型测试和保存

In [9]:
# 设置为评估模式
model.eval() 

ResNet(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (b

In [10]:
# 屏蔽梯度计算
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 88.24 %


In [11]:
# 保存模型
torch.save(model.state_dict(), 'resnet.ckpt')

## Pytorch模型可视化

### 导出ONNX模型

In [14]:
import torch.onnx

# 按照输入格式，设计随机输入
dummy_input =torch.randn(1, 3, 32, 32).cuda()   
# 导出模型
torch.onnx.export(model,dummy_input, 'resnet.onnx',verbose=True, ) 

graph(%0 : Float(1, 3, 32, 32)
      %1 : Float(16, 3, 3, 3)
      %2 : Float(16)
      %3 : Float(16)
      %4 : Float(16)
      %5 : Float(16)
      %6 : Long()
      %7 : Float(16, 16, 3, 3)
      %8 : Float(16)
      %9 : Float(16)
      %10 : Float(16)
      %11 : Float(16)
      %12 : Long()
      %13 : Float(16, 16, 3, 3)
      %14 : Float(16)
      %15 : Float(16)
      %16 : Float(16)
      %17 : Float(16)
      %18 : Long()
      %19 : Float(16, 16, 3, 3)
      %20 : Float(16)
      %21 : Float(16)
      %22 : Float(16)
      %23 : Float(16)
      %24 : Long()
      %25 : Float(16, 16, 3, 3)
      %26 : Float(16)
      %27 : Float(16)
      %28 : Float(16)
      %29 : Float(16)
      %30 : Long()
      %31 : Float(32, 16, 3, 3)
      %32 : Float(32)
      %33 : Float(32)
      %34 : Float(32)
      %35 : Float(32)
      %36 : Long()
      %37 : Float(32, 32, 3, 3)
      %38 : Float(32)
      %39 : Float(32)
      %40 : Float(32)
      %41 : Float(32)
      %42 : Long()
      

### 模型可视化工具：NETRON

有几种方式：
- 安装ONNX客户端
- ONNX有测试网页可以加载显示模型 ：[Netron](https://lutzroeder.github.io/netron/)
- 安装netron服务，可以通过`import netron` 和 `netron.start('model.onnx')`来启动本地查看服务，打开指定端口即可看到。

In [16]:
import netron
#打开服务
netron.start('resnet.onnx')


Stopping http://localhost:8080
Serving 'resnet.onnx' at http://localhost:8080


### 模型可视化结果

![ResNet](connet.png)